<a href="https://colab.research.google.com/github/manojmanivannan/ApacheSparkEssentials/blob/master/SparkStreamByPluralsight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update
!apt-get install -y openjdk-8-jdk-headless scala jq  > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!pip install -q pyspark findspark
!wget https://github.com/tanaikech/goodls/releases/download/v2.0.1/goodls_linux_amd64
!chmod +x goodls_linux_amd64


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,567 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:13 http://pp

In [5]:
!export SPARK_HOME="/content/spark-2.3.1-bin-hadoop2.7"
!export PATH="$PATH:$SPARK_HOME/bin"

In [7]:
!/content/spark-2.3.1-bin-hadoop2.7/sbin/start-master.sh
!grep -oP "MasterUI' on port ([0-9]+)" /content/spark-2.3.1-bin-hadoop2.7/logs/spark--org.apache.spark.deploy.master*.out 

starting org.apache.spark.deploy.master.Master, logging to /content/spark-2.3.1-bin-hadoop2.7/logs/spark--org.apache.spark.deploy.master.Master-1-ef6247d0bcd0.out


In [15]:
# create new token from ngrok
get_ipython().system_raw('./ngrok authtoken 2IeJHLKYs6eB68PKJQM1VkpPu00_5JXQQZW3hofy6Q6vqTX2')
# revoke the token once you close the notebook
get_ipython().system_raw('./ngrok http 8081 &') # check the log above for the port

!sleep 10 && curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://17c4-35-243-227-38.ngrok.io


In [18]:
!/content/spark-2.3.1-bin-hadoop2.7/sbin/start-slave.sh spark://ef6247d0bcd0:7077

starting org.apache.spark.deploy.worker.Worker, logging to /content/spark-2.3.1-bin-hadoop2.7/logs/spark--org.apache.spark.deploy.worker.Worker-1-ef6247d0bcd0.out


In [37]:
!grep -oP "WorkerUI' on port ([0-9]+)" /content/spark-2.3.1-bin-hadoop2.7/logs/spark--org.apache.spark.deploy.worker*.out 

WorkerUI' on port 8082


In [38]:
get_ipython().system_raw('./ngrok http 8082 &') # check the log above for the port

!sleep 10 && curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://17c4-35-243-227-38.ngrok.io


In [42]:
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://17c4-35-243-227-38.ngrok.io","proto":"https","config":{"addr":"http://localhost:8081","inspect":true},"metrics":{"conns":{"count":16,"gauge":0,"rate1":0.0005256539891939742,"rate5":0.005472946314568023,"rate15":0.007933411425998101,"p50":1092103827,"p90":30823051842.9,"p95":31873625110,"p99":31873625110},"http":{"count":21,"rate1":0.0006375987521590698,"rate5":0.007799588786010481,"rate15":0.010616096537004249,"p50":25402751,"p90":161978341.60000008,"p95":212603649.59999996,"p99":215772337}}},{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://17c4-35-243-227-38.ngrok.io","proto":"http","config":{"addr":"http://localhost:8081","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

# Spark session & context
conf = SparkConf().set('spark.ui.port', '4050')
try:
  sc = SparkContext(conf=conf)
except ValueError:
  sc.stop()
  sc = SparkContext(conf=conf)

spark = SparkSession.builder.master('local[2]').getOrCreate()